# Lecture 3 - Part 2 - Julia Types, Methods and Multiple Dispatch

## Julia, The Story So Far

- In lecture 1, we introduced the basics of the Julia programming language, including arrays, functions, and basic flow control.
- In lecture 2, we introduced Pkg.jl, the Julia package manager, and the library DifferentialEquations.jl.
- Today, we'll "remove the training wheels" and discuss Julia's type system in a lot more detail than before.

## Contents
1. [Types](#types)
2. [Composite Types](#composite-types)
3. [Parametric Types](#parametric-types)
4. [Methods and Multiple Dispatch](#methods)
5. [Parametric Methods](#parametric-methods)
6. [Conclusion](#conclusion)
7. [Further Reading](#further-reading)
8. [Next Week](#next-week)

In [1]:
using Pkg
Pkg.activate(".")
Pkg.add(["BenchmarkTools", "AbstractTrees"])

  Activating project at `~/code/TUM-Dynamics-Lecture/lectures/lecture-3`
   Resolving package versions...
  No Changes to `~/code/TUM-Dynamics-Lecture/lectures/lecture-3/Project.toml`
  No Changes to `~/code/TUM-Dynamics-Lecture/lectures/lecture-3/Manifest.toml`


In [3]:
using BenchmarkTools, AbstractTrees

AbstractTrees.children(x::Type) = subtypes(x)  # Useful method for printing type trees

## 1. Types <a class="anchor" id="types"></a>

- Every value in Julia is a first-class object, and **all objects have a type**.
<br>

- In the words of the Julia documentation,


> There is no division between object and non-object values: all values in Julia are true objects having a type that belongs to a single, fully connected type graph, all nodes of which are equally first-class as types.

- What do we mean when we're talking about a fully connected type graph?

In [4]:
print_tree(Number)

Number
├─ MultiplicativeInverse
│  ├─ SignedMultiplicativeInverse
│  └─ UnsignedMultiplicativeInverse
├─ Complex
└─ Real
   ├─ AbstractFloat
   │  ├─ BigFloat
   │  ├─ BFloat16
   │  ├─ Float16
   │  ├─ Float32
   │  └─ Float64
   ├─ AbstractIrrational
   │  └─ Irrational
   ├─ Integer
   │  ├─ Bool
   │  ├─ Signed
   │  │  ├─ BigInt
   │  │  ├─ Int128
   │  │  ├─ Int16
   │  │  ├─ Int32
   │  │  ├─ Int64
   │  │  └─ Int8
   │  └─ Unsigned
   │     ├─ UInt128
   │     ├─ UInt16
   │     ├─ UInt32
   │     ├─ UInt64
   │     └─ UInt8
   └─ Rational


- Why do we need a [type system](https://en.wikipedia.org/wiki/Type_system) at all? Types are essential in specifying the **structure and behaviour** of our program, both at a high level (the code we write in Julia) and at a low level (the underlying actions taken by our computer when we run our code).

In [5]:
typeof(1.0)

Float64

In [6]:
typeof(1)

Int64

In [7]:
bitstring(1.0)

"0011111111110000000000000000000000000000000000000000000000000000"

In [8]:
bitstring(1)

"0000000000000000000000000000000000000000000000000000000000000001"

In [9]:
@code_native 1 + 1

	.text
	.file	"+"
	.globl	"julia_+_11368"                 # -- Begin function julia_+_11368
	.p2align	4, 0x90
	.type	"julia_+_11368",@function
"julia_+_11368":                        # @"julia_+_11368"
; Function Signature: +(Int64, Int64)
; ┌ @ int.jl:87 within `+`
# %bb.0:                                # %top
; │ @ int.jl within `+`
	#DEBUG_VALUE: +:x <- $rdi
	#DEBUG_VALUE: +:y <- $rsi
	push	rbp
	mov	rbp, rsp
; │ @ int.jl:87 within `+`
	lea	rax, [rdi + rsi]
	pop	rbp
	ret
.Lfunc_end0:
	.size	"julia_+_11368", .Lfunc_end0-"julia_+_11368"
; └
                                        # -- End function
	.section	".note.GNU-stack","",@progbits


In [10]:
@code_native 1.0 + 1.0

	.text
	.file	"+"
	.globl	"julia_+_11469"                 # -- Begin function julia_+_11469
	.p2align	4, 0x90
	.type	"julia_+_11469",@function
"julia_+_11469":                        # @"julia_+_11469"
; Function Signature: +(Float64, Float64)
; ┌ @ float.jl:491 within `+`
# %bb.0:                                # %top
; │ @ float.jl within `+`
	#DEBUG_VALUE: +:x <- $xmm0
	#DEBUG_VALUE: +:y <- $xmm1
	push	rbp
	mov	rbp, rsp
; │ @ float.jl:491 within `+`
	vaddsd	xmm0, xmm0, xmm1
	pop	rbp
	ret
.Lfunc_end0:
	.size	"julia_+_11469", .Lfunc_end0-"julia_+_11469"
; └
                                        # -- End function
	.type	".L+Core.Float64#11471",@object # @"+Core.Float64#11471"
	.section	.rodata,"a",@progbits
	.p2align	3, 0x0
".L+Core.Float64#11471":
	.quad	".L+Core.Float64#11471.jit"
	.size	".L+Core.Float64#11471", 8

.set ".L+Core.Float64#11471.jit", 140554016807056
	.size	".L+Core.Float64#11471.jit", 8
	.section	".note.GNU-stack","",@progbits


In [11]:
@code_native 1 + 1.0

	.text
	.file	"+"
	.globl	"julia_+_11473"                 # -- Begin function julia_+_11473
	.p2align	4, 0x90
	.type	"julia_+_11473",@function
"julia_+_11473":                        # @"julia_+_11473"
; Function Signature: +(Int64, Float64)
; ┌ @ promotion.jl:429 within `+`
# %bb.0:                                # %top
; │ @ promotion.jl within `+`
	#DEBUG_VALUE: +:x <- $rdi
	#DEBUG_VALUE: +:y <- $xmm0
	push	rbp
	mov	rbp, rsp
; │ @ promotion.jl:429 within `+`
; │┌ @ promotion.jl:400 within `promote`
; ││┌ @ promotion.jl:375 within `_promote`
; │││┌ @ number.jl:7 within `convert`
; ││││┌ @ float.jl:239 within `Float64`
	vcvtsi2sd	xmm1, xmm1, rdi
; │└└└└
; │ @ promotion.jl:429 within `+` @ float.jl:491
	vaddsd	xmm0, xmm1, xmm0
	pop	rbp
	ret
.Lfunc_end0:
	.size	"julia_+_11473", .Lfunc_end0-"julia_+_11473"
; └
                                        # -- End function
	.type	".L+Core.Float64#11475",@object # @"+Core.Float64#11475"
	.section	.rodata,"a",@progbits
	.p2align	3, 0x0
".L+Core.

- Julia's type system is [**dynamic**](https://en.wikipedia.org/wiki/Type_system#Dynamic_type_checking_and_runtime_type_information). (Also [nominative](https://en.wikipedia.org/wiki/Nominal_type_system) and [parametric](https://en.wikipedia.org/wiki/Parametric_polymorphism).)
<br>

- In a statically typed language, such as C, C++, and Fortran, every variable has a type which must be specified in advance by the programmer. For example, in C, `int a;`.
<br>

- In a dynamically typed language, such as Python and Julia, **values have a type only at runtime**, and do not have to be specified in advance. This can save time for the programmer, and make your code more generic (it can work with many different kinds of runtime types - this is called [polymorphism](https://en.wikipedia.org/wiki/Polymorphism_(computer_science))).
<br>

- **Since Julia is dynamically typed, types are associated with values, not variables.** A variable is simply a name "bound" to a value.

In [12]:
a = 1

1

In [13]:
typeof(a)

Int64

### 1.1. Abstract and Concrete Types

- We said earlier that all Julia types belong to "a single, fully connected type graph, all nodes of which are equally first-class as types."
<br>

- We can further classify the types in the type graph as either:
    1. **Abstract**, or
    2. **Concrete**.<br><br>
- **KEY FACT 1: All Julia objects are instances of a concrete type.**

In [24]:
typeof(1.0)

Float64

In [25]:
isconcretetype(typeof(1.0))

true

In [26]:
is_instance_of_concrete_type(x) = isconcretetype(typeof(x))  # Returns true for all x

is_instance_of_concrete_type (generic function with 1 method)

In [27]:
is_instance_of_concrete_type(Float64)

true

- In turn, all concrete types are subtypes of at least one abstract type.

In [28]:
supertype(typeof(1.0))

AbstractFloat

In [29]:
supertypes(Float64)

(Float64, AbstractFloat, Real, Number, Any)

In [30]:
isabstracttype(AbstractFloat)

true

- The abstract type `Any` sits at the top of the type tree. By definition, all types, both abstract and concrete, are subtypes of `Any`.
<br>

- We can use the operator `<:`, meaning "is a subtype of" to test whether one type is a subtype of another type.

In [34]:
Float64 <: Number

true

In [35]:
is_subtype_of_Any(x) = typeof(x) <: Any  # Returns true for all x

is_subtype_of_Any (generic function with 1 method)

In [36]:
is_subtype_of_Any(Float64)

true

- **KEY FACT 2: Concrete types may not subtype each other.**
<br>

- In other words, **concrete types are final**, and only abstract types may be supertypes.

In [37]:
has_abstract_supertype(x) = isabstracttype(supertype(typeof(x)))  # Returns true for all x

has_abstract_supertype (generic function with 1 method)

- **KEY FACT 3: Abstract types cannot be instantiated.**
<br>

- (This is really the same as Key Fact 1, but it's worth emphasising again.)
<br>

- Therefore, abstract types serve only to describe relationships between related concrete types within Julia's type hierarchy. 
<br>

- Abstract types are branches in the type tree while concrete types are leaves.

In [38]:
print_tree(Number)

Number
├─ MultiplicativeInverse
│  ├─ SignedMultiplicativeInverse
│  └─ UnsignedMultiplicativeInverse
├─ Complex
└─ Real
   ├─ AbstractFloat
   │  ├─ BigFloat
   │  ├─ BFloat16
   │  ├─ Float16
   │  ├─ Float32
   │  └─ Float64
   ├─ AbstractIrrational
   │  └─ Irrational
   ├─ Integer
   │  ├─ Bool
   │  ├─ Signed
   │  │  ├─ BigInt
   │  │  ├─ Int128
   │  │  ├─ Int16
   │  │  ├─ Int32
   │  │  ├─ Int64
   │  │  └─ Int8
   │  └─ Unsigned
   │     ├─ UInt128
   │     ├─ UInt16
   │     ├─ UInt32
   │     ├─ UInt64
   │     └─ UInt8
   └─ Rational


- Why do we even need abstract types then, if they can never be instantiated?
<br>

- A well-defined type hierarchy, used in combination with **methods** and **multiple dispatch**, allows you to define behaviour for entire families of related concrete types. More on this later.

### 1.2. Defining Abstract Types

- You can define your own abstract types using the `abstract type` keyword.

In [39]:
abstract type Animal end

- In this context, we can use the `<:` operator - "is a subtype of" - to indicate relationships between types and thereby construct a user-defined type hierarchy.

In [40]:
abstract type Feline <: Animal end
abstract type Canine <: Animal end

In [41]:
print_tree(Animal)

Animal
├─ Canine
└─ Feline


- Remember, we can't instantiate abstract types, so we still can't create any animals. 


## 2. Composite Types (structs)  <a class="anchor" id="composite-types"></a>

- How can we define our own concrete types? 
<br>

- The most common user-defined type in Julia is a **composite type** or `struct`. This is the closest Julia equivalent of a `Class` in Python.


### 2.1. Defining Composite Types

- Composite types are introduced with the `struct` keyword followed by a block of field names.

In [42]:
struct Foo
    baz
    qux
    quux
end

In [43]:
foo = Foo("a", "b", "c")

Foo("a", "b", "c")

In [44]:
typeof(foo)

Foo

- You can also add type annotations to struct definitions, using the `::` operator (meaning "is a").

In [45]:
struct Bar
    baz  # implicit ::Any
    qux::Float64
    quux::Int64
end

In [46]:
bar = Bar("a", "b", "c")

LoadError: MethodError: [0mCannot `convert` an object of type [92mString[39m[0m to an object of type [91mFloat64[39m
The function `convert` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T<:Number
[0m[90m   @[39m [90mBase[39m [90m[4mnumber.jl:6[24m[39m
[0m  convert(::Type{T}, [91m::T[39m) where T
[0m[90m   @[39m [90mBase[39m [90m[4mBase.jl:126[24m[39m
[0m  convert(::Type{T}, [91m::Number[39m) where T<:Number
[0m[90m   @[39m [90mBase[39m [90m[4mnumber.jl:7[24m[39m
[0m  ...


In [47]:
Bar("a", 1.0, 1)

Bar("a", 1.0, 1)

In [48]:
Bar("a", 1.0, 1.0)

Bar("a", 1.0, 1)

- Doing this can make your code readable, ensure correctness, and has implications for performance (more on this later). However, it can also make your code less flexible if you include overly restrictive type annotations.
<br>

- **Whenever you omit an explicit type annotation on the field of a struct, `::Any` is implied.**
<br>

- As with the definition of abstract types, we can use the `<:` operator - "is a subtype of" - to indicate that a composite type is a subtype of a specific abstract type.

In [49]:
struct Dog <: Canine
    name
    home
end

struct Wolf <: Canine
    name
    pack
end

struct Cat <: Feline
    name
    home
end

struct Lion <: Feline
    name
    pride
end

In [50]:
print_tree(Animal)

Animal
├─ Canine
│  ├─ Dog
│  └─ Wolf
└─ Feline
   ├─ Cat
   └─ Lion


In [51]:
struct Labrador <: Dog
    name
end

LoadError: invalid subtyping in definition of Labrador: can only subtype abstract types.

### 2.2. Instantiating Composite Types

- To create a new object of a given type, simply apply the type name like a function.

In [52]:
sammy = Dog("Sammy", "Schmidt Family")

Dog("Sammy", "Schmidt Family")

In [53]:
fieldnames(Dog)

(:name, :home)

- When we call the type name like this, we are actually calling a **constructor** for that type, which is a function which returns an instance of the given type.
<br>

- Two default constructors are created automatically every time we declare a new composite type:
    1. One which accepts any arguments and attempts to convert them to the types of the fields.
    2. Another which accepts arguments matching the field types exactly.

In [54]:
struct A
    a::Float32
    b::Float64
end

In [55]:
methods(A)

# 2 methods for type constructor:
 [1] A(a::Float32, b::Float64)
     @ In[54]:2
 [2] A(a, b)
     @ In[54]:2

In [58]:
@which A(1.0, 1.0)

A(a, b)
     @ Main In[54]:2

In [59]:
@which A(1f0, 1e0)

A(a::Float32, b::Float64)
     @ Main In[54]:2

### 2.3. Accessing the Fields of a Composite Type

- You can access the field names of a composite type using the standard `foo.bar` notation.

In [60]:
sammy.name

"Sammy"

In [61]:
sammy.home

"Schmidt Family"

### 2.4. Structs are Not Mutable

- Composite objects declared with `struct` are **immutable**; they cannot be modified after construction.

In [62]:
fieldnames(Dog)

(:name, :home)

In [63]:
sammy.name

"Sammy"

In [64]:
sammy.name = "Good Boy"

LoadError: setfield!: immutable struct of type Dog cannot be changed

- **N.B. An immutable object can contain mutable objects, such as arrays, as fields. Those mutable field values will remain mutable; only the fields of the immutable object itself cannot be changed to point to different objects.**

In [65]:
struct ArrayStruct
    a::Array
end
array_struct = ArrayStruct([1.0, 2.0])
array_struct.a

2-element Vector{Float64}:
 1.0
 2.0

In [66]:
# Modify the mutable object
array_struct.a[1] = 3.0
array_struct.a

2-element Vector{Float64}:
 3.0
 2.0

In [67]:
# Attempt to modify the field value itself
array_struct.a = [3.0, 4.0, 5.0]

LoadError: setfield!: immutable struct of type ArrayStruct cannot be changed

- Why is it like this? Isn't this unnecessarily restrictive for the programmer?
<br>

- The main reason for structs to be immutable is to allow the compiler to effectively optimise your code. In particular, it allows your objects to be stored efficiently in memory. (Hopefully) more on this in a later lecture.
<br>

- If you really do need a mutable struct, you can simply declare one with the `mutable` keyword:

In [68]:
mutable struct MyMutableStruct
    a
end
m = MyMutableStruct("original value")
m.a

"original value"

In [69]:
m.a = "new value"
m.a

"new value"

- Be warned that using mutable structs may impact the performance of your program and should be avoided in performance critical code.
<br>

- If you find yourself regularly reaching for mutable structs, it may be because you are still transitioning to the "Julian" way of thinking.

### 2.5. Avoid Abstract Type Annotations

- To enable Julia to compile your source code into highly efficient machine code, it is strongly advised to avoid abstract type annotations in performance critical code. 
<br>

- For example, don't do this:

In [70]:
struct TwoThings
    a  # implicit ::Any
    b
end

function multiply(two_things)
    return two_things.a * two_things.b
end

two_things = TwoThings(3.0, 4.0)

@btime multiply(two_things)

  28.454 ns (1 allocation: 16 bytes)


12.0

- Or this:

In [71]:
struct TwoAbstractFloats
    a::AbstractFloat
    b::AbstractFloat
end

two_floats = TwoAbstractFloats(3.0, 4.0)

@btime multiply(two_floats)

  32.905 ns (1 allocation: 16 bytes)


12.0

- What if we had used concrete type annotations instead?

In [72]:
struct TwoFloat64s
    a::Float64
    b::Float64
end

two_float64s = TwoFloat64s(3.0, 4.0)

@btime multiply(two_float64s)

  17.013 ns (1 allocation: 16 bytes)


12.0

In [73]:
struct TwoFloat32
    a::Float32
    b::Float32
end

- **Even for such a trivial operation, we can see that there is a significant impact on speed due to the abstract type annotations.**
<br>

- However, we also don't want to create a new, distinct composite type for every possible combination of concrete field types. Fortunately, Julia's parametric type system provides convenient syntax for defining entire families of structs with concrete field types.

## 3. Parametric Types <a class="anchor" id="parametric-types"></a>

- We just saw how to define composite types, or structs. For example, to define a point in 2D Cartesian space we could do:

In [74]:
struct PointGeneric
    x
    y
end

- In addition, we saw that we can annotate the struct's fields with types, either abstract or concrete, and that this can have consequences for the performance of our code. For example:

In [75]:
struct PointFloat64
    x::Float64
    y::Float64
end

In [76]:
function norm(point)
    (; x, y) = point  # unpack the fields of a struct using (;)
    return sqrt(x^2 + y^2)
end

norm (generic function with 1 method)

In [83]:
p1 = PointGeneric(3.0, 4.0)
@btime norm(p1)

  94.201 ns (4 allocations: 64 bytes)


5.0

In [84]:
p2 = PointFloat64(3.0, 4.0)
@btime norm(p2)

  21.169 ns (1 allocation: 16 bytes)


5.0

- Creating a new `Point`-like type for every possible concrete type its fields could take is of course tedious and restrictive. Do not write code like this!

### 3.1. Parametric Composite Types 

- Julia's type system is **parametric**: types can take parameters, such that type declarations actually introduce an entire family of new types - one for each possible combination of the parameter values.
<br>

- We can think of this like a mathematical function with parameters, e.g., $f(x, y \,; a, b)$.
<br>

- Type parameters are introduced immediately after the type name, surrounded by curly braces:

In [85]:
struct Point{T}
   x::T
   y::T
end

- `T`, the type parameter, can be *any* Julia type.
<br>

- This particular declaration defines a new parametric type, `Point{T}`, holding two "coordinates", each of the same type `T`.
<br>

- For example, `Point{Float64}` is a concrete type equivalent to the type defined by replacing `T` in the definition of `Point` with `Float64`. That is, `Point{Float64}` is equivalent to the `PointFloat64` struct that we defined earlier.

In [87]:
Point{Float64}

Point{Float64}

In [88]:
isconcretetype(Point{Float64})

true

In [89]:
Point{String}

Point{String}

In [90]:
p3 = Point{Any}(3.0, 4.0)
@btime norm(p3)

  81.679 ns (4 allocations: 64 bytes)


5.0

In [91]:
p4 = Point{Float64}(3.0, 4.0)
@btime norm(p4)

  18.622 ns (1 allocation: 16 bytes)


5.0

In [93]:
isconcretetype(Point{Float64})

true

In [92]:
isabstracttype(Point)

false

In [94]:
isstructtype(Point)

true

- We see therefore that our parametric definition of `Point{T}` actually declares an unlimited number of types, one for each possible value of `T`, each of which is now a usable concrete type. 
<br>

- When creating an instance of `Point`, the value of the parameter type `T` can be omitted if it is unambiguous:

In [95]:
Point(1.0, 2.0)

Point{Float64}(1.0, 2.0)

In [96]:
Point("1.0", "2.0")

Point{String}("1.0", "2.0")

In [97]:
methods(Point)

# 1 method for type constructor:
 [1] Point(x::T, y::T) where T
     @ In[85]:2

- `Point` itself is a valid type object (neither abstract nor concrete!) containing all possible instances of `Point{T}` as subtypes:

In [98]:
Point{Float64} <: Point

true

In [99]:
Point{Any} <: Point

true

- However, concrete `Point` types with different values of `T` are never subtypes of each other:

In [101]:
Point{Float64} <: Point{AbstractFloat}

false

In [100]:
Float64 <: AbstractFloat

true

- This is because Julia's type system is [**invariant**](https://en.wikipedia.org/wiki/Covariance_and_contravariance_(computer_science)), which means that, for any three types `A`, `B`, `C`, where `C` is parametrised by another type, `A <: B` does not imply that `C{A} <: C{B}`.
<br>

- Often, we will want to restrict the possible values of the type parameter `T`. For example, to restrict our coordinates to real numbers, we could have done:

In [102]:
struct PointReal{T<:Real}
   x::T
   y::T
end

In [103]:
PointReal{String}

LoadError: TypeError: in PointReal, in T, expected T<:Real, got Type{String}

In [104]:
String <: Real

false

In [105]:
function count_subtypes(T)
    count = 0
    
    for subtype in subtypes(T)
        count += 1
        if !isconcretetype(subtype)
            count += count_subtypes(subtype)
        end
    end
    
    return count
end

count_subtypes (generic function with 1 method)

In [106]:
count_subtypes(Real)

24

In [107]:
print_tree(Real)

Real
├─ AbstractFloat
│  ├─ BigFloat
│  ├─ BFloat16
│  ├─ Float16
│  ├─ Float32
│  └─ Float64
├─ AbstractIrrational
│  └─ Irrational
├─ Integer
│  ├─ Bool
│  ├─ Signed
│  │  ├─ BigInt
│  │  ├─ Int128
│  │  ├─ Int16
│  │  ├─ Int32
│  │  ├─ Int64
│  │  └─ Int8
│  └─ Unsigned
│     ├─ UInt128
│     ├─ UInt16
│     ├─ UInt32
│     ├─ UInt64
│     └─ UInt8
└─ Rational


- We can also have multiple type parameters for a single parametric type. For example:

In [108]:
struct Person{A<:AbstractString, B<:Integer, C<:Real}
    name::A
    age::B
    height::C
    weight::C
end

In [109]:
Person("Alice", 100, 200.0, 100.0)

Person{String, Int64, Float64}("Alice", 100, 200.0, 100.0)

### 3.2. Parametric Abstract Types

- Parametric abstract type declarations declare a collection of abstract types, in much the same way:

In [110]:
abstract type Pointy{T} end

- Once again, our parametric type declaration defines an unlimited number of abstract types, one for every possible value of `T`.
<br>

- We could have declared `Point{T}` to be a subtype of `Pointy{T}`. For example:

In [111]:
struct Point2D{T} <: Pointy{T}
   x::T
   y::T
end

In [112]:
struct Point3D{T} <: Pointy{T}
   x::T
   y::T
   z::T
end

- We now have a distinct type tree for each allowed value of `T`. For example:

In [113]:
print_tree(Pointy{Float64})

Pointy{Float64}
├─ Point2D{Float64}
└─ Point3D{Float64}


In [114]:
print_tree(Pointy{Int32})

Pointy{Int32}
├─ Point2D{Int32}
└─ Point3D{Int32}


In [115]:
print_tree(Pointy)

Pointy
├─ Point2D
└─ Point3D


## Summary So Far

1. Types in Julia can be abstract or concrete.
<br>

2. A composite type is introduced with the `struct` keyword and consists of a block of field names along with optional type annotations.
<br>

3. Structs are immutable.
<br>

4. Abstract type annotations can slow down your code.
<br>

5. Parametric types define entire families of types.

## 4. Methods and Multiple Dispatch <a class="anchor" id="methods"></a>

### 4.1 Methods


- Remember from the first lecture that a function maps a tuple of arguments to a return value.
<br>

- From the Julia documentation (emphasis mine):


> It is common for the same conceptual function or operation to be implemented quite differently for **different types of arguments**: adding two integers is very different from adding two floating-point numbers, both of which are distinct from adding an integer to a floating-point number. Despite their implementation differences, these operations all fall under the general concept of "addition". Accordingly, in Julia, these behaviors all belong to a single object: the + function.


- Conceptually, we have one function (addition), with many possible behaviours, depending on the types of the inputs (integers, floats, matrices, and so on).
<br>

- **A definition of one possible behaviour for a function, given the number and types of its arguments, is called a method.**

In [116]:
methods(+)

# 198 methods for generic function "+" from Base:
   [1] +(a::Pkg.Resolve.VersionWeight, b::Pkg.Resolve.VersionWeight)
     @ Pkg.Resolve ~/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/share/julia/stdlib/v1.11/Pkg/src/Resolve/versionweights.jl:22
   [2] +(B::BitMatrix, J::LinearAlgebra.UniformScaling)
     @ LinearAlgebra ~/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/share/julia/stdlib/v1.11/LinearAlgebra/src/uniformscaling.jl:151
   [3] +(x::Bool, z::Complex{Bool})
     @ complex.jl:308
   [4] +(x::Bool, y::Bool)
     @ bool.jl:166
   [5] +(x::Bool)
     @ bool.jl:163
   [6] +(x::Bool, z::Complex)
     @ complex.jl:315
   [7] +(x::Real, z::Complex{Bool})
     @ complex.jl:322
   [8] +(x::Bool, y::T) where T<:AbstractFloat
     @ bool.jl:173
   [9] +(z::Complex{Bool}, x::Bool)
     @ complex.jl:309
  [10] +(z::Complex{Bool}, x::Real)
     @ complex.jl:323
  [11] +(z::Complex, x::Bool)
     @ complex.jl:316
  [12] +(x::BigInt, y::BigInt)
     @ Base.GMP gmp.jl:496
  [13] +(a::BigInt, b::BigInt, c::BigInt)
     @ Base.GMP gmp.jl:536
  [14] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt)
     @ Base.GMP gmp.jl:537
  [15] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt)
     @ Base.GMP gmp.jl:538
  [16] +(x::BigInt, y::BigInt, rest::BigInt...)
     @ Base.GMP gmp.jl:678
  [17] +(c::BigInt, x::BigFloat)
     @ Base.MPFR mpfr.jl:527
  [18] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8})
     @ Base.GMP gmp.jl:544
  [19] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8})
     @ Base.GMP gmp.jl:550
  [20] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat)
     @ Base.MPFR mpfr.jl:678
  [21] +(x::BigFloat, y::BigFloat)
     @ Base.MPFR mpfr.jl:491
  [22] +(a::BigFloat, b::BigFloat, c::BigFloat)
     @ Base.MPFR mpfr.jl:665
  [23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat)
     @ Base.MPFR mpfr.jl:671
  [24] +(x::BigFloat, c::BigInt)
     @ Base.MPFR mpfr.jl:522
  [25] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8})
     @ Base.MPFR mpfr.jl:498
  [26] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8})
     @ Base.MPFR mpfr.jl:506
  [27] +(x::BigFloat, c::Union{Float16, Float32, Float64})
     @ Base.MPFR mpfr.jl:514
  [28] +(x::Rational{BigInt}, y::Rational{BigInt})
     @ Base.GMP.MPQ gmp.jl:1056
  [29] +(a::Pkg.Resolve.FieldValue, b::Pkg.Resolve.FieldValue)
     @ Pkg.Resolve ~/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/share/julia/stdlib/v1.11/Pkg/src/Resolve/fieldvalues.jl:43
  [30] +(level::Base.CoreLogging.LogLevel, inc::Integer)
     @ Base.CoreLogging logging/logging.jl:132
  [31] +(t::Dates.Time, dt::Dates.Date)
     @ Dates ~/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:22
  [32] +(x::Dates.Time, y::Dates.TimePeriod)
     @ Dates ~/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:85
  [33] +(::Missing, ::Missing)
     @ missing.jl:122
  [34] +(::Missing)
     @ missing.jl:101
  [35] +(x::Missing, y::Dates.AbstractTime)
     @ Dates ~/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:91
  [36] +(::Missing, ::Number)
     @ missing.jl:123
  [37] +(dt::Dates.DateTime, y::Dates.Year)
     @ Dates ~/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:25
  [38] +(dt::Dates.DateTime, z::Dates.Month)
     @ Dates ~/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:49
  [39] +(x::Dates.DateTime, y::Dates.Quarter)
     @ Dates ~/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:77
  [40] +(x::Dates.DateTime, y::Dates.Period)
     @ Dates ~/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/share/julia/stdlib/v1.11/Dates/src/arithmetic.jl:83
  [41] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod)
     @ Dates ~/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/share/julia/stdlib/v1.11/Dates/src/periods.jl:335
  [42] +(x::Dates.CompoundPeriod, y

### 4.2. Multiple Dispatch

- Given a single function with many methods, how does Julia choose which method to call?

> The choice of which method to execute when a function is applied is called dispatch. **Julia allows the dispatch process to choose which of a function's methods to call based on the number of arguments given, and on the types of all of the function's arguments**. This is different than traditional object-oriented languages, where dispatch occurs based only on the first argument, which often has a special argument syntax, and is sometimes implied rather than explicitly written as an argument. **Using all of a function's arguments to choose which method should be invoked, rather than just the first, is known as multiple dispatch**. 


- In particular, Julia will automatically select the **most specific** method matching the arguments provided.
<br>

- What do we mean by most specific? Multiple dispatch can work on both abstract types and concrete types, and subtypes are considered more specific than their supertypes. 
<br>

- **This is why a well-defined type hierarchy and multiple dispatch go hand-in-hand.**


### 4.3. Defining Methods


- To define new methods, simply use the same function definition syntax we saw in the first lecture:<br>
    1. The first time you define a function with a given name, the function object will be created along with a single method.
    2. Each subsequent function declaration with the same function name and distinct arguments (distinct by either number or type) will create a new method associated with the existing function.

In [117]:
function add(x, y)
    x + y
end

add (generic function with 1 method)

In [118]:
methods(add)

# 1 method for generic function "add" from Main:
 [1] add(x, y)
     @ In[117]:1

In [119]:
function add(x::Float64, y::Float64)
    x + y
end

add (generic function with 2 methods)

In [120]:
methods(add)

# 2 methods for generic function "add" from Main:
 [1] add(x::Float64, y::Float64)
     @ In[119]:1
 [2] add(x, y)
     @ In[117]:1

In [121]:
function add(x::Float64, y::Float32)
    x + y
end

add (generic function with 3 methods)

In [122]:
methods(add)

# 3 methods for generic function "add" from Main:
 [1] add(x::Float64, y::Float32)
     @ In[121]:1
 [2] add(x::Float64, y::Float64)
     @ In[119]:1
 [3] add(x, y)
     @ In[117]:1

In [123]:
function add(x, y, z)
    x + y + z
end

add (generic function with 4 methods)

In [124]:
methods(add)

# 4 methods for generic function "add" from Main:
 [1] add(x::Float64, y::Float32)
     @ In[121]:1
 [2] add(x::Float64, y::Float64)
     @ In[119]:1
 [3] add(x, y, z)
     @ In[123]:1
 [4] add(x, y)
     @ In[117]:1

In [125]:
add(1, 2, 3)

6

In [127]:
@which add(1, 2f0)

add(x, y)
     @ Main In[117]:1

- **Important: unlike type annotations on structs, type annotations do not have impact on the performance of methods**. 
<br>

- Instead, each time you call a function with a new combination of arguments for the first time, Julia will automatically compile an efficient version of that function for those specific argument types. Each subsequent invocation of the function with the same argument types will use the already compiled method.

In [128]:
methods(add)

# 4 methods for generic function "add" from Main:
 [1] add(x::Float64, y::Float32)
     @ In[121]:1
 [2] add(x::Float64, y::Float64)
     @ In[119]:1
 [3] add(x, y, z)
     @ In[123]:1
 [4] add(x, y)
     @ In[117]:1

In [130]:
@btime add(1f0, 2f0)  # 32-bit floats

  1.231 ns (0 allocations: 0 bytes)


3.0f0

In [131]:
function add(x::Float32, y::Float32)
    x + y
end

add (generic function with 5 methods)

In [132]:
methods(add)

# 5 methods for generic function "add" from Main:
 [1] add(x::Float32, y::Float32)
     @ In[131]:1
 [2] add(x::Float64, y::Float32)
     @ In[121]:1
 [3] add(x::Float64, y::Float64)
     @ In[119]:1
 [4] add(x, y, z)
     @ In[123]:1
 [5] add(x, y)
     @ In[117]:1

In [133]:
@btime add(1f0, 2f0)

  1.193 ns (0 allocations: 0 bytes)


3.0f0

### 4.4. Example: Animals

- Earlier, we defined some animals. Now, let's add behaviour using methods and multiple dispatch.

In [134]:
print_tree(Animal)

Animal
├─ Canine
│  ├─ Dog
│  └─ Wolf
└─ Feline
   ├─ Cat
   └─ Lion


In [135]:
fieldnames(Dog)

(:name, :home)

In [136]:
fieldnames(Wolf)

(:name, :pack)

In [137]:
fieldnames(Cat)

(:name, :home)

In [138]:
fieldnames(Lion)

(:name, :pride)

In [139]:
# Create some animals
teddy = Dog("Teddy", "Fischer Family")
yukon = Wolf("Yukon", "Inner Alaska Pack")
felix = Cat("Felix", "Müller Family")
simba = Lion("Simba", "Mount Elgon Pride");

In [140]:
# Methods can be annotated with abstract types
function encounter(canine_one::Canine, canine_two::Canine)
    println("Uncertain whether these two canines will get along. Remain wary.")
end

# Methods without type annotations will accept any type
function encounter(animal_one, animal_two)
    println("No well-defined encounter for a $(typeof(animal_one)) and a $(typeof(animal_two)).")
end

encounter (generic function with 2 methods)

In [141]:
methods(encounter)

# 2 methods for generic function "encounter" from Main:
 [1] encounter(canine_one::Canine, canine_two::Canine)
     @ In[140]:2
 [2] encounter(animal_one, animal_two)
     @ In[140]:7

In [142]:
encounter(teddy, yukon)

Uncertain whether these two canines will get along. Remain wary.


In [143]:
encounter(teddy, felix)

No well-defined encounter for a Dog and a Cat.


In [144]:
# Define more specific methods annotated with concrete types
function encounter(dog_one::Dog, dog_two::Dog)
    println("$(dog_one.name) wags tail. So does $(dog_two.name).")
end

function encounter(cat_one::Cat, cat_two::Cat)
    println("No thanks.")
end

function encounter(dog::Dog, cat::Cat)
    println("$(cat.name) chases $(dog.name) away.")
end

encounter(cat::Cat, dog::Dog) = encounter(dog, cat)  # Allow arguments to be provided in either order

encounter (generic function with 6 methods)

In [145]:
methods(encounter)

# 6 methods for generic function "encounter" from Main:
 [1] encounter(cat::Cat, dog::Dog)
     @ In[144]:14
 [2] encounter(dog::Dog, cat::Cat)
     @ In[144]:10
 [3] encounter(cat_one::Cat, cat_two::Cat)
     @ In[144]:6
 [4] encounter(dog_one::Dog, dog_two::Dog)
     @ In[144]:2
 [5] encounter(canine_one::Canine, canine_two::Canine)
     @ In[140]:2
 [6] encounter(animal_one, animal_two)
     @ In[140]:7

In [146]:
encounter(teddy, felix)

Felix chases Teddy away.


In [147]:
encounter(simba, felix)

No well-defined encounter for a Lion and a Cat.


In [148]:
encounter(felix, felix)

No thanks.


- **Multiple dispatch is one of the core programming paradigms of the Julia language.**
<br>

- How does this compare to standard object-oriented languages? How would you do same example with the animals and their encounters in Python?
<br>


- **Tip**: To get the most out of Julia's multiple dispatch, avoid the temptation to define overly specific functions. For example, you could define a method `encounter` for *any* objects which can do a thing called encounter. Add methods to the function `encounter` instead of defining new functions like `encounter_dog_dog` or `encounter_dog_dog_dog`.

## 5. Parametric Methods <a class="anchor" id="parametric-methods"></a>


- Type parameters can also be used in method definitions. For example:

In [1]:
function f(x::T, y::T) where {T}
    return true
end

function f(x, y)
    return false
end

f (generic function with 2 methods)

- What do you think this function does?


In [5]:
f("a", "b")

true

In [6]:
f(1, "a")

false

In [4]:
f(1e0, 1f0)

false

In [2]:
methods(f)

# 2 methods for generic function "f" from Main:
 [1] f(x::T, y::T) where T
     @ In[1]:1
 [2] f(x, y)
     @ In[1]:5

- As before, the values of `T` can be restricted to subtypes of a given type. For example:

In [7]:
function f_numeric(x::T, y::T) where {T<:Number}
    return true
end

function f_numeric(x, y)
    return false
end

f_numeric (generic function with 2 methods)

- What do you think this function does?

In [8]:
methods(f_numeric)

# 2 methods for generic function "f_numeric" from Main:
 [1] f_numeric(x::T, y::T) where T<:Number
     @ In[7]:1
 [2] f_numeric(x, y)
     @ In[7]:5

In [11]:
f_numeric(1e0, 1e0)

true

In [12]:
f_numeric("a", "b")

false

- The method type parameter `T` can also be used inside the function body:

In [13]:
function g(x::T) where {T}
    return T
end

g (generic function with 1 method)

- What do you think this function does?

In [14]:
g(1f0)

Float32

- As before, multiple type parameters are possible:

In [15]:
function concat_number_to_string(x::S, y::N) where {S<:AbstractString, N<:Number}
    return x * string(y)
end

function concat_number_to_string(x::N, y::S) where {N<:Number, S<:AbstractString}
    return string(x) * y
end

concat_number_to_string (generic function with 2 methods)

In [16]:
concat_number_to_string("My number is ", 1.0)

"My number is 1.0"

In [17]:
concat_number_to_string(1.0, " is my number.")

"1.0 is my number."

In [18]:
methods(concat_number_to_string)

# 2 methods for generic function "concat_number_to_string" from Main:
 [1] concat_number_to_string(x::N, y::S) where {N<:Number, S<:AbstractString}
     @ In[15]:5
 [2] concat_number_to_string(x::S, y::N) where {S<:AbstractString, N<:Number}
     @ In[15]:1

- Why are parametric methods useful? One use-case is to ensure the **correctness** of your code, by enforcing invariants.

In [19]:
function strict_add(x::T, y::T) where {T}
    x + y
end

strict_add (generic function with 1 method)

In [21]:
Float32(1)

1.0f0

In [27]:
Int8(1)

1

In [25]:
a = 1

1

In [26]:
a::Int8

LoadError: TypeError: in typeassert, expected Int8, got a value of type Int64

In [20]:
strict_add(1f0, 2e0)

LoadError: MethodError: no method matching strict_add(::Float32, ::Float64)
The function `strict_add` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  strict_add(::T, [91m::T[39m) where T
[0m[90m   @[39m [35mMain[39m [90m[4mIn[19]:1[24m[39m


In [ ]:
@code_lowered 1f0 + 1f0

In [ ]:
@code_lowered 1.0f0 + 2.0e0

## 6. Conclusion <a class="anchor" id="conclusion"></a>

- When you need to define custom structure and behaviour in Julia - which will be often - think about:<br>
    1. Structs (for structure).
    2. Methods which operate on your structs (for behaviour).

## 7. Further Reading <a class="anchor" id="further-reading"></a>

- The [official Julia documentation](https://docs.julialang.org/en/v1/) is quite thorough and well-explained on the topics we've covered today:  

    1. [Types](https://docs.julialang.org/en/v1/manual/types/)
    2. [Methods and Multiple Dispatch](https://docs.julialang.org/en/v1/manual/methods/)
    3. [Julia Performance Tips](https://docs.julialang.org/en/v1/manual/performance-tips/)<br><br>
    

- [JuliaCon 2019 | The Unreasonable Effectiveness of Multiple Dispatch | Stefan Karpinski](https://www.youtube.com/watch?v=kc9HwsxE1OY)


## 8. Next Week <a class="anchor" id="next-week"></a>
1. Chaos in Dynamical Systems, Part I
2. [DynamicalSystems.jl](https://juliadynamics.github.io/DynamicalSystems.jl/latest/), Part I